# Code move the data of the rodent basal ganglia data set from CSV files to a Neo4j database

To set up the graph, you need to add a dotenv file with the bolt url, username and password for your Neo4j database.
In addition, all python packages have to be installed. Se the README for more details on this.

To run all the code, either select `Cell -> Run All` or use Voila.

To use the data for analytics, all ascii charactes have to be removed. In this case set the variable `remove_ascii` to `True` below.

This is set up tp work on Neo4j 4.0.1 database and only for the use of analytics. Thereby there are no similar analyses and no neuromorpho nodes.

In [358]:
from neo4j import GraphDatabase, basic_auth
from dotenv import load_dotenv
import os

load_dotenv()

neo4jUser = os.getenv("NEO4J_USER")
neo4jPwd = os.getenv("NEO4J_PASSWORD")
neo4jUrl = os.getenv("NEO4j_BOLT")

driver = GraphDatabase.driver(neo4jUrl,auth=basic_auth(neo4jUser, neo4jPwd), encrypted = False)

Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("10054; 'An existing connection was forcibly closed by the remote host'; None; 10054; None")
Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("10054; 'An existing connection was forcibly closed by the remote host'; None; 10054; None")


In [359]:
remove_ascii = True

def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)
    
def get_csv_file_path(file_name):
    folder = "basal_ganglia_valid_ascii" if remove_ascii else "basal_ganglia"
    return "https://raw.githubusercontent.com/marenpg/jupyter_basal_ganglia/master/Data/csvs/%s/%s" % (folder, file_name)


## Adding all nodes

### CELLS
This section wipes all the graph data about cell types and adds the cell type nodes from the CSVs

In [360]:
# Deleting all cell data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:CellType 
                OR n:CellClass
                OR n:CellGroup
                OR n:CellPhenotype 
                OR n:CellPhenotypeCategory 
                OR n:CellularRegion
                OR n:CellDescription
                OR n:NeuralStructure
            DETACH DELETE n
        ''')

        print("wiped database - cells")

wiped database - cells


In [361]:
# Adding CellType to graph from cell_types.csv

csv_file_path = get_csv_file_path("cells/cell_types.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellType { id: row.ID, name: row.Cell_type_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    

    
    session.run(query)
    get_count_of_type("CellType", session)

Added 103 nodes of type CellType


In [362]:
# Adding CellClass to graph from cell_classes.csv

csv_file_path = get_csv_file_path("cells/cell_classes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellClass { id: row.ID, name: row.Cell_class_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    
    
    session.run(query)
    get_count_of_type("CellClass", session)

Added 8 nodes of type CellClass


In [363]:
# Adding CellGroup to graph from cell_types.csv

csv_file_path = get_csv_file_path("cells/cell_groups.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellGroup { id: row.ID, name: row.Cell_group_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    
    
    session.run(query)
    get_count_of_type("CellGroup", session)

Added 3 nodes of type CellGroup


In [364]:
# Adding CellPhenotype from cell_phenotypes.csv

csv_file_path = get_csv_file_path("cells/cell_phenotypes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotype { id: row.ID, name: row.Phenotype, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("CellPhenotype", session)

Added 107 nodes of type CellPhenotype


In [365]:
# Adding CellPhenotypeCategory from cell_phenotype_categories.csv

csv_file_path = get_csv_file_path("cells/cell_phenotype_categories.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotypeCategory { id:row.ID, name: row.Phenotype_category})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("CellPhenotypeCategory", session)

Added 7 nodes of type CellPhenotypeCategory


In [366]:
# Adding CellularRegion from cellular_regions.csv

csv_file_path = get_csv_file_path("cells/cellular_regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellularRegion { id:row.ID, name: row.Cellular_region, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("CellularRegion", session)

Added 10 nodes of type CellularRegion


In [367]:
# Adding CellDescription from cell_descriptions.csv

csv_file_path = get_csv_file_path("cells/cell_description.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellDescription { id: row.id, description: row.description, iri: row.iri, name: row.iri})
    """ % csv_file_path

with driver.session() as session:
    
    session.run(query)
    get_count_of_type("CellDescription", session)

Added 12 nodes of type CellDescription


In [368]:
# Adding NeuralStructure from objects_of_intrest.csv

csv_file_path = get_csv_file_path("cells/objects_of_interest.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:NeuralStructure { id:row.ID, name: row.Object_of_interest, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("NeuralStructure", session)

Added 14 nodes of type NeuralStructure


### Experiments
This section wipes all the graph data about experiment and adds the experiment related nodes from the CSVs

In [369]:
# Deleting all cell data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Calculation
                OR n:StereologyDetail
                OR n:Distribution
                OR n:CellMorphology 
                OR n:Quantitation
                OR n:Analysis
                OR n:Experiment
                OR n:SectioningInstrument
                OR n:Reporter
                OR n:ReporterLabel
                OR n:ReporterTarget
                OR n:ReporterType
                OR n:VisualizationProtocol
                OR n:Solution
                OR n:Software
                OR n:Microscope
                OR n:ElectronMicroscopeDetail
                OR n:LightFluorescenceMicroscopeDetail
                OR n:IntroductionTiming
                OR n:IntroductionType
                OR n:TreatmentPurpose
            DETACH DELETE n
        ''')

        print("wiped database - expeiments")

wiped database - expeiments


In [370]:
# Adding Calculation to graph from calculations.csv
csv_file_path = get_csv_file_path("experiments/calculations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Calculation { id: row.ID, description: row.Calculation_performed, name: row.Calculation_performed})
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_type("Calculation", session)

Added 958 nodes of type Calculation


In [371]:
# Adding StereologyDetail to graph from stereology_details.csv
csv_file_path = get_csv_file_path("experiments/stereology_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:StereologyDetail { id: row.ID, name: row.Stereology_details_name, probe: row.Probe, identificationFeature: row.Identification_feature, disectorHeight: toInteger(row.Disector_height), areaSubfraction: toFloat(row.Area_subfraction), heightSubfraction: toFloat(row.Height_subfraction), investigatedSections: toFloat(row.Investigated_sections), investigatedFields: toInteger(row.Investigated_fields), countedObjects: toInteger(row.Counted_objects), coefficientOfError: toFloat(row.Coefficient_of_error), estimatedVolume: toFloat(row.Estimated_volume), volumeUnit: row.Volume_unit, anyExceptProbe: toInteger(row.any_except_probe)})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("StereologyDetail", session)

Added 212 nodes of type StereologyDetail


In [372]:
# Adding Distribution to graph from distributions.csv
csv_file_path = get_csv_file_path("experiments/distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Distribution:DataType { id: row.ID, name: row.Distribution_name, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, distribution: row.Distribution, distributionDimensions: row.Distribution_dimensions, analysisTypePrimary: row.Analysis_type_primary, analysisTypeSecondary: row.Analysis_type_secondary})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("Distribution", session)

Added 18 nodes of type Distribution


In [373]:
# Adding CellMorphology to graph from cell_morphologies.csv
csv_file_path = get_csv_file_path("experiments/cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellMorphology:DataType { id: row.ID, name: row.Morphology_name, neuromorphoId: row.Neuromorpho_ID, somaSurface: toFloat(row.Soma_surface), numberOfStems: toInteger(row.Number_stems), numberOfBifurcations: toInteger(row.Number_bifurcations), numberOfBranches: toInteger(row.Number_branches), overallWidth: toFloat(row.Overall_width), overallHeight: toFloat(row.Overall_height), overallDepth: toFloat(row.Overall_depth), averageBranchDiameter: toFloat(row.Average_branch_diameter), averageContraction: toFloat(row.Average_contraction), totalArborLength: toFloat(row.Total_arbor_length), totalArborSurface: toFloat(row.Total_arbor_surface), totalArborVolume: toFloat(row.Total_arbor_volume), maxEuclideanDistance: toFloat(row.Max_Euclidean_distance), maxPathDistance: toFloat(row.Max_path_distance), maxBranchOrder: toInteger(row.Max_branch_order), totalFragmentation: toInteger(row.Total_fragmentation), partitionAsymmetry: toFloat(row.Partition_asymmetry), averageRalls: toInteger(row.Average_Ralls), averageBifurcationAngleLocal: toFloat(row.Average_bifurcation_angle_local), averageBifurcationAngleRemote: toFloat(row.Average_bifurcation_angle_remote), fractalDimension: toFloat(row.Fractal_dimension), physicalIntegrity: row.Physical_integrity, structuralDomains: row.Structural_domains, morphologicalAttributes: row.Morphological_attributes, originalFormat: row.Original_format})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("CellMorphology", session)

Added 50 nodes of type CellMorphology


In [374]:
# Adding Quantitation to graph from cell_morphologies.csv
csv_file_path = get_csv_file_path("experiments/quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Quantitation:DataType { id: row.ID, name: row.Quantitation_name, estimateRelevance: row.Estimate_relevance, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, originalExtent: row.Original_extent, number: row.Number, numberSD: row.Number_SD, density: row.Density, densityUnit: row.Density_unit, densitySD: row.Density_SD, volumetricDensity: toInteger(row.Volumetric_density), estimateExtraction: row.Estimate_extraction})
    """ % csv_file_path

with driver.session() as session: 
    
    
    session.run(query)
    get_count_of_type("Quantitation", session)

Added 1074 nodes of type Quantitation


In [375]:
# Adding Analysis to graph from derived_data_records.csv
csv_file_path = get_csv_file_path("experiments/derived_data_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Analysis { id: row.ID, name: row.Derived_data_record_name, numberOfAnimals: row.Number_of_animals, dataType: row.Derived_data_type, shrinkageCorrection: row.Shrinkage_correction })
    """ % csv_file_path

with driver.session() as session:    
    
    
    session.run(query)
    get_count_of_type("Analysis", session)

Added 668 nodes of type Analysis


In [376]:
# Adding Experiment to graph from experiments.csv
csv_file_path = get_csv_file_path("experiments/experiments.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Experiment { id: row.ID, name: row.Experiment_name, animalStatus: row.Animal_status, ageLowerLimit: toInteger(row.Age_lower_limit), ageUpperLimit: toInteger(row.Age_upper_limit), weightLowerLimit: toInteger(row.Weight_lower_limit), weightUpperLimit: toInteger(row.Weight_upper_limit) })
    """ % csv_file_path

with driver.session() as session: 
    
    
    session.run(query)
    get_count_of_type("Experiment", session)

Added 375 nodes of type Experiment


In [377]:
# Adding SectioningInstrument to graph from sectioning_instruments.csv
csv_file_path = get_csv_file_path("experiments/sectioning_instruments.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SectioningInstrument { id: row.ID, name: row.Sectioning_instrument_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SectioningInstrument", session)

Added 7 nodes of type SectioningInstrument


In [378]:
# Adding Reporter to graph from reporters.csv
csv_file_path = get_csv_file_path("experiments/reporters.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Reporter { id: row.ID, name: row.Reporter_name, type: row.Reporter_type, rrid: row.Reporter_unique_ID, comment: row.Comment, originSpecie: row.Origin_species  })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Reporter", session)

Added 249 nodes of type Reporter


In [379]:
# Adding ReporterLabel to graph from reporter_labels.csv
csv_file_path = get_csv_file_path("experiments/reporter_labels.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterLabel { id: row.ID, name: row.Label})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterLabel", session)

Added 23 nodes of type ReporterLabel


In [380]:
# Adding ReporterTarget to graph from reporter_targets.csv
csv_file_path = get_csv_file_path("experiments/reporter_targets.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterTarget { id: row.ID, phenotype: row.Phenotype, name: row.Phenotype})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterTarget", session)

Added 100 nodes of type ReporterTarget


In [381]:
# Adding VisualizationProtocol to graph from visualization_protocols.csv
csv_file_path = get_csv_file_path("experiments/visualization_protocols.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:VisualizationProtocol { id: row.ID, name: row.Protocol_name, ontology: row.Ontological_identifier })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("VisualizationProtocol", session)

Added 12 nodes of type VisualizationProtocol


In [382]:
# Adding Solution to graph from solutions.csv
csv_file_path = get_csv_file_path("experiments/solutions.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Solution { id: row.ID, name: row.Solution_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Solution", session)

Added 266 nodes of type Solution


In [383]:
# Adding Microscope to graph from microscopes.csv
csv_file_path = get_csv_file_path("experiments/microscopes.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Microscope { id: row.ID, type: row.Microscope_type, ontology: row.Ontological_identifier, name: row.Microscope_type })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Microscope", session)

Added 9 nodes of type Microscope


In [384]:
# Adding ElectronMicroscopeDetail to graph from electron_microscopy_details.csv
csv_file_path = get_csv_file_path("experiments/electron_microscopy_details.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ElectronMicroscopeDetail { id: row.ID, name: row.EM_details_name, gridType: row.Grid_type, magnification: row.Magnification })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ElectronMicroscopeDetail", session)

Added 27 nodes of type ElectronMicroscopeDetail


In [385]:
# Adding LightFluorescenceMicroscopeDetail to graph from light_fluorescence_microscopy_details.csv
csv_file_path = get_csv_file_path("experiments/light_fluorescence_microscopy_details.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:LightFluorescenceMicroscopeDetail { id: row.ID, name: row.Microscopy_details_name, refractionMedium: row.Refraction_medium, numericalAperature: toFloat(row.Numerical_aperture), objectiveLens: row.Objective_lens, totalMagnification: row.Total_magnification, pixelSize: toFloat(row.Pixel_size), z_stack: toBoolean(row.Z_stack), opticalSliceSize: row.Optical_slice_size })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("LightFluorescenceMicroscopeDetail", session)

Added 238 nodes of type LightFluorescenceMicroscopeDetail


In [386]:
# Adding Software to graph from softwares.csv
csv_file_path = get_csv_file_path("experiments/softwares.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Software { id: row.ID, name: row.Software_name, rrid: row.Software_RRID })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Software", session)

Added 50 nodes of type Software


In [387]:
# Adding introduction types and timings from introduction_timings.csv and introduction_types.csv
csv_file_path = get_csv_file_path("experiments/introduction_timings.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:IntroductionTiming { id: row.id, name: row.name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("IntroductionTiming", session)

csv_file_path = get_csv_file_path("experiments/introduction_types.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:IntroductionType { id: row.id, name: row.name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("IntroductionType", session)

Added 2 nodes of type IntroductionTiming
Added 6 nodes of type IntroductionType


In [388]:
# Adding treatment purpose types and timings from treatment_purposes.csv
csv_file_path = get_csv_file_path("experiments/treatment_purposes.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:TreatmentPurpose { id: row.id, name: row.name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("TreatmentPurpose", session)

Added 36 nodes of type TreatmentPurpose


### Brain regions
This section wipes all the graph data about brain regions and adds the brain region related nodes from the CSVs

In [389]:
# Deleting all region data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:BrainRegion 
                OR n:RegionZone 
                OR n:RegionRecord 
                OR n:Nomenclature
                OR n:RegionOther
                OR n:BamsRegion
            DETACH DELETE n
            
        ''')

        print("wiped database - brain regions")

wiped database - brain regions


In [390]:
# Adding regions to graph with BrainRegion label
csv_file_path = get_csv_file_path("regions/regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (d:BrainRegion {id: row.ID, name: row.Region_name, abbreviation: row.Abbreviation, comments: row.Comments})
    """ % csv_file_path

with driver.session() as session:    
    
    
    session.run(query)
    get_count_of_type("BrainRegion", session)

Added 88 nodes of type BrainRegion


In [391]:
# Adding region_records to graph with RegionRecord label
csv_file_path = get_csv_file_path("regions/region_records.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:RegionRecord { id: row.ID, name: row.Region_record_name, coverage: row.Coverage, specificity: row.Specificity, numberOfOriginalRegions: toInteger(row.No_original_regions), originalRegionRetained: toInteger(row.Original_region_retained), parcellationScheme: row.Parcellation_scheme, atlasCoordinates: row.Atlas_coordinates, illustration: row.Illustration, semanticDescription: row.Semantic_description, annotatedImages: row.Annotated_images, regionalCharacteristics: row.Regional_characteristics, isAtlasRegion: row.Atlas_reg, serialSections: row.Serial_sections, collectorsComment: row.Collectors_comment, documentationScore: toInteger(row.Documentation_score)})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("RegionRecord", session)

Added 313 nodes of type RegionRecord


In [392]:
# Adding region_zones to graph with RegionZone label
csv_file_path = get_csv_file_path("regions/region_zones.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:RegionZone { id: row.ID, name: row.Region_zone, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("RegionZone", session)

Added 79 nodes of type RegionZone


In [393]:
# Adding nomenclatures to graph with Nomenclature label
csv_file_path = get_csv_file_path("regions/nomenclatures.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Nomenclature { id: row.ID, name: row.Nomenclature_name, version: row.Version, authors: row.Authors, published: toInteger(row.Published), publication_type: row.Publication_type, doi: row.DOI, preferred: 1})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("Nomenclature", session)

Added 3 nodes of type Nomenclature


In [394]:
# Adding nomebclatures_other to graph with Nomenclature label
csv_file_path = get_csv_file_path("regions/nomenclatures_other.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Nomenclature { id: row.ID, name: row.Nomenclature_name, version: row.Version, authors: row.Authors, published: toInteger(row.Published), publicationType: row.Publication_type, preferred: 0 })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Nomenclature", session)

Added 19 nodes of type Nomenclature


In [395]:
# Adding regions_other to graph with RegionOther label
csv_file_path = get_csv_file_path("regions/regions_other.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (d:RegionOther {id: row.ID, name: row.Region_name, abbreviation: row.Abbreviation, comments: row.Comments})
    """ % csv_file_path

with driver.session() as session:    
    
    
    session.run(query)
    get_count_of_type("RegionOther", session)

Added 272 nodes of type RegionOther


In [396]:
# Adding bams2_regions to graph with BamsRegion label
csv_file_path = get_csv_file_path("regions/bams2_regions.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (d:BamsRegion {id: row.id, name: row.name, description: row.desciption})
    """ % csv_file_path

with driver.session() as session:    
    
    
    session.run(query)
    get_count_of_type("BamsRegion", session)

Added 80 nodes of type BamsRegion


### Sources
This section wipes all the graph data about sources and adds the nodes related to sources from the CSVs

In [397]:
# Deleting all source data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Source
                OR n:SourceOrigin
                OR n:ConsideredPaper
                OR n:ExclusionReason
            DETACH DELETE n
        ''')

        print("wiped database - sources")

wiped database - sources


In [398]:
# Adding Source to graph from sources.csv

csv_file_path = get_csv_file_path("sources/sources.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Source {id: row.ID, title: row.Source_title, type: row.Source_type, insertedDate: row.Inserted_date, publicationYear: toInteger(row.Source_publication_year), sourceName: row.Source_name, rawDataAvailable: row.Raw_data_available})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("Source", session)

Added 246 nodes of type Source


In [399]:
# Adding SourceOrigin to graph from sources.csv

csv_file_path = get_csv_file_path("sources/source_origins_lookup.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SourceOrigin {id: row.ID, name: row.Source_name, identifier: row.Identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SourceOrigin", session)

Added 363 nodes of type SourceOrigin


In [400]:
# Adding ConsideredPaper to graph from sources.csv
## TODO try to connect considered_paper and publication 
## based on first auth + year (published) with Source_name

csv_file_path = get_csv_file_path("sources/considered_papers.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ConsideredPaper {id: row.ID, title: row.Title, publishedYear: toInteger(row.Published), firstAuthor: row.First_author, isIncluded: false, name: row.Title})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    
    get_count_of_type("ConsideredPaper", session)
    


included_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Decision =~ ".*(?i)Included.*"
        MATCH (n:ConsideredPaper {id: row.Paper})
        SET n.isIncluded = true
        RETURN COUNT(n)
    """ % csv_file_path

with driver.session() as session:    
    res = session.run(included_query)
    print("Number of considered papers that are included:",  res.value()[0])

Added 2204 nodes of type ConsideredPaper
Number of considered papers that are included: 0


In [401]:
### Process considered paper decisions to find reasons where considered paper can get a relation to the reaseon.
## IF it doesnt have relation to excluded it  is included 

import pandas as pd
csv_file_path = get_csv_file_path("sources/considered_papers_desicions.csv")

df = pd.read_csv(csv_file_path)

exclusion_reasons = df["Decision"].unique()

with driver.session() as session: 
    for reason in exclusion_reasons:
        if type(reason) == str and "included" not in reason.lower():
            reason_text = reason.replace("Excluded: ", "")
            q = "CREATE (:ExclusionReason {name: '%s', reason: '%s'})" % (reason_text, reason_text)
            session.run(q)
            
            # Relationship EXCLUSION_REASON between ConsideredPaper and ExclusionReason
            query="""
                    LOAD CSV WITH HEADERS FROM "%s" AS row
                    WITH row where row.Decision = '%s'
                    MATCH (a:ConsideredPaper { id: row.Paper})
                    MATCH (c:ExclusionReason { reason: '%s' })
                    MERGE (a)-[:EXCLUSION_REASON]->(c)
                """ % (csv_file_path, reason,  reason.replace("Excluded: ", ""))

            session.run(query)
    get_count_of_type("ExclusionReason", session)
    get_count_of_relationship("EXCLUSION_REASON", session)



Added 13 nodes of type ExclusionReason
Added 4226 relationships of type EXCLUSION_REASON


### Specimens
This section wipes all the graph data about specimens and adds the specimen related nodes from the CSVs

In [402]:
# Deleting all subject data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Specimen
                OR n:Specie 
                OR n:Strain 
                OR n:Substrain 
                OR n:Sex
                OR n:AgeCategory
                OR n:TransgenicLine
            DETACH DELETE n
        ''')

        print("wiped database - specimens")

wiped database - specimens


In [403]:
# Adding Specimen to graph from specimens.csv

csv_file_path = get_csv_file_path("subjects/specimens.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Specimen {id: row.ID, name: row.Specimen_name, form: row.Specimen_form, order: row.Specimen_order})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("Specimen", session)

Added 1065 nodes of type Specimen


In [404]:
# Adding Strain to graph from strains.csv

csv_file_path = get_csv_file_path("subjects/strains.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Strain {id: row.ID, name: row.Strain_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("Strain", session)

Added 24 nodes of type Strain


In [405]:
# Adding Substrain to graph from substrains.csv

csv_file_path = get_csv_file_path("subjects/substrains.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Substrain {id: row.ID, name: row.Substrain_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("Substrain", session)

Added 44 nodes of type Substrain


In [406]:
# Adding Specie to graph from species.csv

csv_file_path = get_csv_file_path("subjects/species.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Specie {id: row.ID, name: row.Species, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    
    
    session.run(query)
    get_count_of_type("Specie", session)

Added 2 nodes of type Specie


In [407]:
# Adding AgeCategory to graph from age_categories.csv

csv_file_path = get_csv_file_path("subjects/age_categories.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:AgeCategory {id: row.ID, name: row.Age_category, ontology: row.Ontological_identifier, description: row.Description})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("AgeCategory", session)

Added 10 nodes of type AgeCategory


In [408]:
# Adding Sex to graph from sex.csv

csv_file_path = get_csv_file_path("subjects/sex.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Sex {id: row.ID, name: row.Sex, ontology: row.Ontology_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Sex", session)

Added 3 nodes of type Sex


In [409]:
# Adding TransgenicLine to graph from transgenic_lines.csv

csv_file_path = get_csv_file_path("subjects/transgenic_lines.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:TransgenicLine {id: row.ID, name: row.Transgenic_line_name, RRID: row.Transgenic_line_RRID})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("TransgenicLine", session)

Added 20 nodes of type TransgenicLine


## Relationships
With all the nodes added to the graph, we add the relationships between them.

### Specimen relationships
This section creates relationships between specimen related nodes other nodes in the graph from CSVs and node information.

In [410]:
# Relationship EXPERIMENT_SPECIMEN between Experiment to Specimen

csv_file_path = get_csv_file_path("subjects/specimens.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.Experiment_ID})
        MATCH (c:Specimen { id: row.ID })
        MERGE (a)-[:EXPERIMENT_SPECIMEN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("EXPERIMENT_SPECIMEN", session)

Added 2122 relationships of type EXPERIMENT_SPECIMEN


In [411]:
# Relationship :HAS_SUBSTRAIN between Strain to Substrain

csv_file_path = get_csv_file_path("subjects/substrains.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Strain { id: row.Strain_name})
        MATCH (c:Substrain { id: row.ID })
        MERGE (a)-[:HAS_SUBSTRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("HAS_SUBSTRAIN", session)

Added 66 relationships of type HAS_SUBSTRAIN


In [412]:
# Relationship :FOR_SPECIE between AgeCategory to Specie

csv_file_path = get_csv_file_path("subjects/age_categories.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:AgeCategory { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:FOR_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("FOR_SPECIE", session)

Added 20 relationships of type FOR_SPECIE


### Cell relationships
This section creates relationships between cell related nodes other nodes in the graph from CSVs and node information.

In [413]:
# Relationship CELL_GROUP_MEMBERSHIP between CellClass and CellGroup

csv_file_path = get_csv_file_path("cells/cell_classes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellClass { id: row.ID})
        MATCH (c:CellGroup { id: row.Cell_group_membership })
        MERGE (a)-[:CELL_GROUP_MEMBERSHIP]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_GROUP_MEMBERSHIP", session)

Added 16 relationships of type CELL_GROUP_MEMBERSHIP


In [414]:
# Relationship CELL_CLASS_MEMBERSHIP between CellType and CellClass

csv_file_path = get_csv_file_path("cells/cell_types.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellType { id: row.ID})
        MATCH (c:CellClass { id: row.Cell_class_membership })
        MERGE (a)-[:CELL_CLASS_MEMBERSHIP]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_CLASS_MEMBERSHIP", session)

Added 206 relationships of type CELL_CLASS_MEMBERSHIP


In [415]:
# Relationship PHENOTYPE_CATEGORY between CellPhenotype and CellPhenotypeCategory

csv_file_path = get_csv_file_path("cells/cell_phenotypes.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellPhenotype { id: row.ID})
        MATCH (c:CellPhenotypeCategory { id: row.Phenotype_category })
        MERGE (a)-[:PHENOTYPE_CATEGORY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PHENOTYPE_CATEGORY", session)

Added 210 relationships of type PHENOTYPE_CATEGORY


In [416]:
# Relationship PHENOTYPE between Cell and CellPhenotype

csv_file_path = get_csv_file_path("cells/celltype_phenotype.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellType { id: row.CellID})
        MATCH (c:CellPhenotype { id: row.PhenotypeID })
        MERGE (a)-[:PHENOTYPE]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PHENOTYPE", session)

Added 322 relationships of type PHENOTYPE


In [417]:
# Relationship CELL_DESCRIPTION between CellType, CellClass, CellGroup and CellDescription

csv_file_path = get_csv_file_path("cells/cell_description.csv")

type_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellType { id: row.cell_type_id})
        MATCH (c:CellDescription { id: row.id })
        MERGE (a)-[:CELL_DESCRIPTION]->(c)
    """ % csv_file_path

class_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellClass { id: row.cell_class_id})
        MATCH (c:CellDescription { id: row.id })
        MERGE (a)-[:CELL_DESCRIPTION]->(c)
    """ % csv_file_path

group_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellGroup { id: row.cell_group_id})
        MATCH (c:CellDescription { id: row.id })
        MERGE (a)-[:CELL_DESCRIPTION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(type_query)
    session.run(class_query)
    session.run(group_query)
    get_count_of_relationship("CELL_DESCRIPTION", session)

Added 32 relationships of type CELL_DESCRIPTION


### Expreiment relationships
This section creates relationships between experiment related nodes other nodes in the graph from CSVs and node information.

In [418]:
def get_count_of_relationship(rel_label, fromLabel, toLabel, session):
    q = "MATCH (:%s)-[r:%s]-(:%s) RETURN count(*)" % (fromLabel, rel_label, toLabel)
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", fromLabel, rel_label, toLabel)

In [419]:
# Relationship BELONGS_TO between NeuralStructure and Cell

csv_file_path = get_csv_file_path("experiments/derived_data_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:NeuralStructure { id: row.Object_of_interest})
        MATCH (c:CellType { id: row.Cell_type_putative })
        MERGE (a)-[:BELONGS_TO]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("BELONGS_TO","NeuralStructure","CellType", session)

Added 134 relationships of type NeuralStructure BELONGS_TO CellType


In [420]:
## Microscope RELATIONS

# :ElectronMicroscopeDetail - :MICROSCOPE_TYPE - :Microscope
csv_file_path = get_csv_file_path("experiments/electron_microscopy_details.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:ElectronMicroscopeDetail { id: row.ID})
        MATCH (c:Microscope { id: row.Microscope })
        MERGE (a)-[:MICROSCOPE_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE_TYPE", "ElectronMicroscopeDetail", "Microscope", session)
    
# :LightFluorescenceMicroscopeDetail - :MICROSCOPE_TYPE - :Microscope
csv_file_path = get_csv_file_path("experiments/light_fluorescence_microscopy_details.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:LightFluorescenceMicroscopeDetail { id: row.ID})
        MATCH (c:Microscope { id: row.Microscope })
        MERGE (a)-[:MICROSCOPE_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE_TYPE", "LightFluorescenceMicroscopeDetail", "Microscope", session)
    
# :LightFluorescenceMicroscopeDetail - :MOUNTING_MEDIUM - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:LightFluorescenceMicroscopeDetail { id: row.ID})
        MATCH (c:Solution { id: row.Mounting_medium })
        MERGE (a)-[:MOUNTING_MEDIUM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MOUNTING_MEDIUM", "LightFluorescenceMicroscopeDetail", "Solution", session)

# :Analysis - :MICROSCOPE - :ElectronMicroscopeDetail
csv_file_path = get_csv_file_path("experiments/derived_data_EMdetails.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:ElectronMicroscopeDetail { id: row.EMdetails })
        MERGE (a)-[:MICROSCOPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE", "Analysis", "ElectronMicroscopeDetail", session)

# :Analysis - :MICROSCOPE - :LightFluorescenceMicroscopeDetail
csv_file_path = get_csv_file_path("experiments/derived_data_LFMdetails.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record_ID})
        MATCH (c:LightFluorescenceMicroscopeDetail { id: row.LFMdetails_ID })
        MERGE (a)-[:MICROSCOPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE", "Analysis", "LightFluorescenceMicroscopeDetail", session)


Added 27 relationships of type ElectronMicroscopeDetail MICROSCOPE_TYPE Microscope
Added 238 relationships of type LightFluorescenceMicroscopeDetail MICROSCOPE_TYPE Microscope
Added 238 relationships of type LightFluorescenceMicroscopeDetail MOUNTING_MEDIUM Solution
Added 59 relationships of type Analysis MICROSCOPE ElectronMicroscopeDetail
Added 538 relationships of type Analysis MICROSCOPE LightFluorescenceMicroscopeDetail


In [421]:
## Reporter RELATIONS
csv_file_path = get_csv_file_path("experiments/reporters.csv")


## :Reporter - :LABEL - :ReporterLabel
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterLabel { id: row.Label })
        MERGE (a)-[:LABEL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("LABEL", "Reporter", "ReporterLabel", session)
    
## :Reporter - :TARGET - :ReporterTarget
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterTarget { id: row.Target })
        MERGE (a)-[:TARGET]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET", "Reporter", "ReporterTarget", session)
    
## :Reporter - :TYPE - :ReporterType
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (r:Reporter {id: row.ID})
        MATCH (t:ReporterType)
        WHERE row.Reporter_type = t.name
        MERGE (a)-[:TYPE]-(t)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TYPE", "Reporter", "ReporterType", session)

Added 246 relationships of type Reporter LABEL ReporterLabel
Added 241 relationships of type Reporter TARGET ReporterTarget
Added 0 relationships of type Reporter TYPE ReporterType


In [422]:
### Experiment RELATIONS
csv_file_path = get_csv_file_path("experiments/experiments.csv")


## :Experiment - :ANAESTHETIC - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Anaesthetic })
        MERGE (a)-[:ANAESTHETIC]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ANAESTHETIC", "Experiment", "Solution", session)
    
## :Experiment - :PERFUSION_FIX_MEDIUM - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Perfusion_fix_medium })
        MERGE (a)-[:PERFUSION_FIX_MEDIUM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFUSION_FIX_MEDIUM", "Experiment", "Solution", session)

## :Experiment - :DATA_ORIGIN - :Source
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Source { id: row.Source })
        MERGE (a)-[:DATA_ORIGIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_ORIGIN", "Experiment", "Source", session)


Added 313 relationships of type Experiment ANAESTHETIC Solution
Added 312 relationships of type Experiment PERFUSION_FIX_MEDIUM Solution
Added 375 relationships of type Experiment DATA_ORIGIN Source


In [423]:
### Analysis RELATIONS 
csv_file_path = get_csv_file_path("experiments/derived_data_records.csv")

## :Analysis - :SPECIMEN - :Specimen 
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:Specimen { id: row.Specimen })
        MERGE (a)-[:SPECIMEN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIMEN", "Analysis", "Specimen", session)
    
## :Analysis - :OBJECT_OF_INTEREST - :NeuralStructure
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:NeuralStructure { id: row.Object_of_interest })
        MERGE (a)-[:OBJECT_OF_INTEREST]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("OBJECT_OF_INTEREST", "Analysis", "NeuralStructure", session)
    
## Add property to :Analysis - :OBJECT_OF_INTEREST - :NeuralStructure
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis {id: row.ID})-[r:OBJECT_OF_INTEREST]-(b:NeuralStructure {id: row.Object_of_interest})
        WHERE row.OOI_recognition_criteria IS NOT NULL
        SET r.recognitionCriteria = row.OOI_recognition_criteria
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
## :Analysis - :CELL_TYPE_PUTATIVE - :Cell
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:CellType { id: row.Cell_type_putative })
        MERGE (a)-[:CELL_TYPE_PUTATIVE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_TYPE_PUTATIVE", "Analysis", "CellType", session)
    
## :Analysis - :VISUALIZATION_METHOD - :VisualizationProtocol
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:VisualizationProtocol { id: row.Visualization_method })
        MERGE (a)-[:VISUALIZATION_METHOD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("VISUALIZATION_METHOD", "Analysis", "VisualizationProtocol", session)


Added 668 relationships of type Analysis SPECIMEN Specimen
Added 668 relationships of type Analysis OBJECT_OF_INTEREST NeuralStructure
Added 668 relationships of type Analysis CELL_TYPE_PUTATIVE CellType
Added 666 relationships of type Analysis VISUALIZATION_METHOD VisualizationProtocol


In [424]:
### Quantitation RELATIONS

## :Analysis - :DATA_TYPE - :Quantitation
csv_file_path = get_csv_file_path("experiments/quantitations.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Quantitation { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Quantitation", session)
    
## :Quantitation - :REGION_RECORD - :RegionRecord

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Quantitation", "RegionRecord", session)
    
## :Quantitation - :REGION_ZONE - :RegionZone
csv_file_path = get_csv_file_path("experiments/quantitations.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "Quantitation", "RegionZone", session)
    
## :Quantitation - :CELLULAR_TARGET_REGION - :CellularRegion
csv_file_path = get_csv_file_path("experiments/quantitations.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_target_region })
        MERGE (a)-[:CELLULAR_TARGET_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_TARGET_REGION", "Quantitation", "CellularRegion", session)
    
## :Quantitation - :TARGET_CELL - :Cell
csv_file_path = get_csv_file_path("experiments/quantitations.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:CellType { id: row.Cellular_target_ID })
        MERGE (a)-[:TARGET_CELL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET_CELL", "Quantitation", "CellType", session)

## :Quantitation - :STEREOLOGY - :StereologyDetail
csv_file_path = get_csv_file_path("experiments/quantitations.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Quantitation", "StereologyDetail", session)
    
## :Quantitation - :SOFTWARE - :Software
csv_file_path = get_csv_file_path("experiments/quantitations.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:SOFTWARE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SOFTWARE", "Quantitation", "Software", session)
    
## :Quantitation - :CALCULATION - :Calculation
csv_file_path = get_csv_file_path("experiments/calculations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.Quantitation_ID})
        MATCH (c:Calculation { id: row.ID })
        MERGE (a)-[:CALCULATION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CALCULATION", "Quantitation", "Calculation", session)

Added 1074 relationships of type Analysis DATA_TYPE Quantitation
Added 1074 relationships of type Quantitation REGION_RECORD RegionRecord
Added 453 relationships of type Quantitation REGION_ZONE RegionZone
Added 102 relationships of type Quantitation CELLULAR_TARGET_REGION CellularRegion
Added 11 relationships of type Quantitation TARGET_CELL CellType
Added 418 relationships of type Quantitation STEREOLOGY StereologyDetail
Added 658 relationships of type Quantitation SOFTWARE Software
Added 958 relationships of type Quantitation CALCULATION Calculation


In [425]:
### CellMorphology RELATIONS

## :Analysis - :DATA_TYPE - :CellMorphology

csv_file_path = get_csv_file_path("experiments/cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:CellMorphology { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "CellMorphology", session)

## :CellMorphology - :REGION_RECORD - :RegionRecord
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "CellMorphology", "RegionRecord", session)
    
## :CellMorphology - :REGION_ZONE - :RegionZone
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "CellMorphology", "RegionZone", session)
    
## :CellMorphology - :RECONSTRUCTED_WITH - :Software
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:Software { id: row.Reconstruction_method })
        MERGE (a)-[:RECONSTRUCTED_WITH]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RECONSTRUCTED_WITH", "CellMorphology", "Software", session)

Added 50 relationships of type Analysis DATA_TYPE CellMorphology
Added 50 relationships of type CellMorphology REGION_RECORD RegionRecord
Added 37 relationships of type CellMorphology REGION_ZONE RegionZone
Added 50 relationships of type CellMorphology RECONSTRUCTED_WITH Software


In [426]:
### DISTRIBUTION RELATIONS

## :Analysis - :DATA_TYPE - :Distribution
csv_file_path = get_csv_file_path("experiments/distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Distribution { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Distribution", session)

## :Distribution - :PERFORMED_ON - :Software 
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:PERFORMED_ON]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFORMED_ON", "Distribution", "Software", session)
    
## :Distribution - :CELLULAR_REGION - :CellularRegion
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_region })
        MERGE (a)-[:CELLULAR_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
# Add property to :relation
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Cellular_region_cytochemical_ID is not null
        MATCH (a:Distribution {id: row.ID})-[r:CELLULAR_REGION]-(b:CellularRegion)
        SET r.cytochemicalId = row.Cellular_region_cytochemical_ID
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_REGION", "Distribution", "CellularRegion", session)

## :Distribution - :REGION_RECORD - :RegionRecord
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Distribution", "RegionRecord", session)

## :Distribution - :STEREOLOGY - :StereologyDetail
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Distribution", "StereologyDetail", session)
    
## :Distribution - :RELATED_TO - :Quantitation
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Quantitation { id: row.Related_quantitation })
        MERGE (a)-[:RELATED_TO]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RELATED_TO", "Distribution", "Quantitation", session)



Added 18 relationships of type Analysis DATA_TYPE Distribution
Added 10 relationships of type Distribution PERFORMED_ON Software
Added 4 relationships of type Distribution CELLULAR_REGION CellularRegion
Added 18 relationships of type Distribution REGION_RECORD RegionRecord
Added 4 relationships of type Distribution STEREOLOGY StereologyDetail
Added 16 relationships of type Distribution RELATED_TO Quantitation


In [427]:
### Derived relations from Experiments and Analysis (DDR) and Specimen

## :Analysis - :SECTIONING_DETAIL - :SectioningInstrument
csv_file_path = get_csv_file_path("/experiments/derived_ddr_sectioning_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:SectioningInstrument { id: row.Sectioning_instrument })
        MERGE (a)-[:SECTIONED_BY {thikness: coalesce(toFloat(row.Section_thickness), ""), orientation: coalesce(row.Section_orientation, "")} ]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECTIONED_BY", "Analysis", "SectioningInstrument", session)

    
## :Analysis - :INCUBATION - :Reporter
csv_file_path = get_csv_file_path("/experiments/derived_ddr_reporters.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:Reporter { id: row.reporterId })
        MERGE (a)-[:USED_REPORTER {order: coalesce(row.Order, ""), concentration: coalesce(row.Concentration, ""), time: coalesce(row.Time, ""), temperature: coalesce(row.temperature, "")}]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("USED_REPORTER", "Analysis", "Reporter", session)

    
## :Analysis - :PART_OF_EXPERIMENT - :Experiment
csv_file_path = get_csv_file_path("experiments/derived_ddr_experiments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:Experiment { id: row.experimentId })
        MERGE (a)-[:PART_OF_EXPERIMENT]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF_EXPERIMENT", "Analysis", "Experiment", session)



Added 610 relationships of type Analysis SECTIONED_BY SectioningInstrument
Added 901 relationships of type Analysis USED_REPORTER Reporter
Added 668 relationships of type Analysis PART_OF_EXPERIMENT Experiment


### REGION RELATIONSHIPS

In [428]:
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)


In [429]:
# Relationship PRIMARY_REGION between BrainRegion and RegionRecord
csv_file_path = get_csv_file_path("regions/region_records.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:BrainRegion { id: row.Region})
        MERGE (a)-[:PRIMARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PRIMARY_REGION", session)

Added 624 relationships of type PRIMARY_REGION


In [430]:
# Relationship SECONDARY_REGION between BrainRegion and RegionRecord
csv_file_path = get_csv_file_path("regions/region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:BrainRegion { id: row.Secondary_region })
        MERGE (a)-[:SECONDARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECONDARY_REGION", session)

Added 58 relationships of type SECONDARY_REGION


In [431]:
#Relationship ORIGINAL_REGION from RegionRecord to RegionOther
csv_file_path = get_csv_file_path("regions/original_region_records.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.Region_record})
        MATCH (c:RegionOther { id: row.Original_region })
        MERGE (a)-[:ORIGINAL_REGION]->(c)
    """ % csv_file_path

rel_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord {id: row.Region_record})-[r:ORIGINAL_REGION]->(b:RegionOther)
        SET r.originalCoordinateMinAP = row.Original_coord_AP_min
        SET r.originalCoordinateMaxAP = row.Original_coord_AP_max
        SET r.originalCoordinateMinDV = row.Original_coord_DV_min
        SET r.originalCoordinateMaxDV = row.Original_coord_DV_max
        SET r.originalCoordinateMinML = row.Original_coord_ML_min
        SET r.originalCoordinateMaxML = row.Original_coord_ML_max
""" % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ORIGINAL_REGION", session)
    session.run(rel_query)
    
# TODO CHECKK HERE

Added 620 relationships of type ORIGINAL_REGION


In [432]:
# Relationship NOMENCLATURE_SPECIE between Nomenclature and Specie
csv_file_path = get_csv_file_path("regions/nomenclatures.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:NOMENCLATURE_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
csv_file_path = get_csv_file_path("regions/nomenclatures_other.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:NOMENCLATURE_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NOMENCLATURE_SPECIE", session)

Added 38 relationships of type NOMENCLATURE_SPECIE


In [433]:
# Relationship NOMENCLATURE_STRAIN between Nomenclature and Strain
csv_file_path = get_csv_file_path("regions/nomenclatures.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:NOMENCLATURE_STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
csv_file_path = get_csv_file_path("regions/nomenclatures_other.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:NOMENCLATURE_STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NOMENCLATURE_STRAIN", session)

Added 34 relationships of type NOMENCLATURE_STRAIN


In [434]:
# Relationship NAMING between RegionOther and Nomenclature
csv_file_path = get_csv_file_path("regions/regions_other.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionOther { id: row.ID})
        MATCH (c:Nomenclature { id: row.Nomenclature })
        MERGE (a)-[:NAMING]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NAMING", session)
    
# TODO CHECKK HERE

Added 544 relationships of type NAMING


In [435]:
# Relationship NAMING between BrainRegion and Nomenclature
csv_file_path = get_csv_file_path("regions/regions.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:Nomenclature { id: row.Nomenclature })
        MERGE (a)-[:NAMING]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NAMING", session)
    
# Relationship REGION_OTHER_NAMING between All RegionOther and Nomenclature
csv_file_path = get_csv_file_path("regions/regions_other.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionOther { id: row.ID})
        MATCH (c:Nomenclature { id: row.Nomenclature })
        MERGE (a)-[:REGION_OTHER_NAMING]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_OTHER_NAMING", session)

Added 720 relationships of type NAMING
Added 544 relationships of type REGION_OTHER_NAMING


In [436]:
# Relationship ORIGINAL_FRAMEWORK between Regions_records and Nomenclatures_other
csv_file_path = get_csv_file_path("regions/region_records.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:Nomenclature { id: row.Original_framework })
        MERGE (a)-[:ORIGINAL_FRAMEWORK]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ORIGINAL_FRAMEWORK", session)

Added 574 relationships of type ORIGINAL_FRAMEWORK


In [437]:
### Add Hierarchical relations between regions
# RAT:
csv_file_path = get_csv_file_path("regions/region_hierarchy_waxholm.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:BrainRegion { id: row.Part_of })
        MERGE (a)-[:PART_OF]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF", session)
    
# MOUSE:
csv_file_path = get_csv_file_path("regions/region_hierarchy_amba.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:BrainRegion { id: row.Part_of })
        MERGE (a)-[:PART_OF]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF", session)


Added 34 relationships of type PART_OF
Added 66 relationships of type PART_OF


### SOURCES RELATIONSHIPS

In [438]:
def get_count_of_relationship(rel_label, fromLabel, toLabel, session):
    q = "MATCH (:%s)-[r:%s]-(:%s) RETURN count(*)" % (fromLabel, rel_label, toLabel)
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", fromLabel, rel_label, toLabel)


In [439]:
# Relationship COLLECTED_FROM between Source and SourceOrigin
csv_file_path = get_csv_file_path("sources/sources.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Source { id: row.ID})
        MATCH (c:SourceOrigin { id: row.Source_origin })
        MERGE (a)-[:COLLECTED_FROM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("COLLECTED_FROM", "Source", "SourceOrigin", session)

Added 245 relationships of type Source COLLECTED_FROM SourceOrigin


In [440]:
# Relationship COLLECTED_FROM between ConsideredPaper and SourceOrigin
csv_file_path = get_csv_file_path("sources/considered_papers.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:ConsideredPaper { id: row.ID})
        MATCH (c:SourceOrigin { id: row.Journal })
        MERGE (a)-[:COLLECTED_FROM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("COLLECTED_FROM", "ConsideredPaper", "SourceOrigin", session)

Added 2134 relationships of type ConsideredPaper COLLECTED_FROM SourceOrigin


### SUBJECT RELATIONSHIPS

In [441]:
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

In [442]:
# Relationship SPECIE between Specimen and Specie
csv_file_path = get_csv_file_path("subjects/derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIE", session)

Added 1006 relationships of type SPECIE


In [443]:
# Relationship STRAIN between Specimen and Strain
csv_file_path = get_csv_file_path("subjects/derived_ddr_specimen.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STRAIN", session)

Added 888 relationships of type STRAIN


In [444]:
# Relationship SUBSTRAIN between Specimen and Specie
csv_file_path = get_csv_file_path("subjects/derived_ddr_specimen.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Substrain { id: row.Substrain })
        MERGE (a)-[:SUBSTRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SUBSTRAIN", session)

Added 240 relationships of type SUBSTRAIN


In [445]:
# Relationship TRANSGENIC_LINE between Specimen and TransgenicLine
csv_file_path = get_csv_file_path("subjects/derived_ddr_specimen.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:TransgenicLine { id: row.Transgenic_line_name })
        MERGE (a)-[:TRANSGENIC_LINE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TRANSGENIC_LINE", session)

Added 78 relationships of type TRANSGENIC_LINE


In [446]:
# Relationship SEX between Specimen and Sex
csv_file_path = get_csv_file_path("subjects/derived_ddr_specimen.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Sex { id: row.Sex })
        MERGE (a)-[:SEX]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SEX", session)

Added 796 relationships of type SEX


In [447]:
# Relationship AGE_CATEGORY between Specimen and AgeCategory
csv_file_path = get_csv_file_path("subjects/derived_ddr_specimen.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:AgeCategory { id: row.ageId })
        MERGE (a)-[:AGE_CATEGORY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("AGE_CATEGORY", session)

Added 900 relationships of type AGE_CATEGORY


In [448]:
# Relationship FROM_SPECIE between Strain and Specie
csv_file_path = get_csv_file_path("subjects/strains.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Strain { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:FROM_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("FROM_SPECIE", session)


Added 48 relationships of type FROM_SPECIE


In [449]:
# Relationship VISUALIZATION_MEDIUM from Specimen to Solution
csv_file_path = get_csv_file_path("experiments/single_cell_labelings.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.Specimen})
        MATCH (c:Solution { id: row.Visualization_medium })
        MERGE (a)-[:VISUALIZATION_MEDIUM {concentration: coalesce(row.Medium_concentration, ""), volume: coalesce(row.Medium_volume, ""), introductionTiming: coalesce(row.Introduction_timing, ""), introductionType: coalesce(row.Introduction_type, "")}]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("VISUALIZATION_MEDIUM", session)
    

# Relationship TREATMENT from Specimen to Solution
csv_file_path = get_csv_file_path("experiments/specimen_treatments.csv")

query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.Specimen_ID})
        MATCH (c:Solution { id: row.Solution })
        MERGE (a)-[:TREATMENT {purpose: coalesce(row.Purpose, ""), time: coalesce(row.Time, ""), timeUnit: coalesce(row.Time_unit, ""), temperature: coalesce(row.Temperature, "")}]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TREATMENT", session)


Added 44 relationships of type VISUALIZATION_MEDIUM
Added 5438 relationships of type TREATMENT


In [450]:
## Mostly for analytics
# Relationship SPECIMEN_TREATMENT_PURPOSE from Analysis to TreatmentPurpose
query="""
        MATCH (a:Analysis)-->(:Specimen)-[r:TREATMENT]->(:Solution)
        MATCH (t:TreatmentPurpose)
        WHERE r.purpose = t.name
        MERGE (a)-[:SPECIMEN_TREATMENT_PURPOSE]-(t)
    """ 

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIMEN_TREATMENT_PURPOSE", session)

Added 3480 relationships of type SPECIMEN_TREATMENT_PURPOSE


In [451]:
## Mostly for analytics
# Relationship CELL_LABELING_USED from Analysis to IntroductionType

query="""
        MATCH (a:Analysis)-->(:Specimen)-[r:VISUALIZATION_MEDIUM]->(:Solution)
        MATCH (i:IntroductionType)
        WHERE r.introductionType = i.name
        MERGE (a)-[:CELL_LABELING_USED]-(i)
    """ 

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_LABELING_USED", session)


# Relationship CELL_LABELING_USED from Analysis to IntroductionTiming
query="""
        MATCH (a:Analysis)-->(:Specimen)-[r:VISUALIZATION_MEDIUM]->(:Solution)
        MATCH (i:IntroductionTiming)
        WHERE r.introductionTiming = i.name
        MERGE (a)-[:CELL_LABELING_USED]-(i)
    """

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_LABELING_USED", session)


Added 34 relationships of type CELL_LABELING_USED
Added 70 relationships of type CELL_LABELING_USED


### EXTENDED RELATIONSHIPS
Adding relationships based on other created relationships in the graph.

In [452]:
# Relationship CELLS_IN_REGION between BrainRegion and Cell

query="""
        MATCH (b:BrainRegion)<-[:PRIMARY_REGION]-(r:RegionRecord)<-[:REGION_RECORD]-(d)<-[:DATA_TYPE]-(:Analysis)-[:CELL_TYPE_PUTATIVE]->(c:CellType)
        MERGE (b)-[:CELLS_IN_REGION]->(c)
    """

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLS_IN_REGION", session)

Added 570 relationships of type CELLS_IN_REGION


In [453]:
# Relationship OBSERVED_IN from RegionZone to BrainRegion
query="""
        MATCH (a:BrainRegion)<-[x]-(b:RegionRecord)<-[:REGION_RECORD]-(c)-[:REGION_ZONE]->(d:RegionZone)
        MERGE (a)<-[:OBSERVED_IN]-(d)
    """

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("OBSERVED_IN", session)

Added 218 relationships of type OBSERVED_IN


In [454]:
# Relationship REGION_SPECIE between BrainRegion and Specie
query="""
        MATCH (a:BrainRegion)-[:NAMING]->(b:Nomenclature)-[:NOMENCLATURE_SPECIE]->(c:Specie)
        MERGE (a)-[:REGION_SPECIE]->(c)
    """

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_SPECIE", session)

Added 176 relationships of type REGION_SPECIE


In [455]:
# Relationship OBSERVED_CELL_REGION from CellType to CellularRegion

query="""
        MATCH (a:CellType)<-[:CELL_TYPE_PUTATIVE]-(b:Analysis)-[:DATA_TYPE]-(c)-[:CELLULAR_TARGET_REGION]->(d:CellularRegion) 
        MERGE (a)-[:OBSERVED_CELL_REGION]->(d)
    """

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("OBSERVED_CELL_REGION", session)

Added 40 relationships of type OBSERVED_CELL_REGION


### Connectivity information from BAMS
This adds information about connectivity in regaion, collected from bams: https://bams2.bams1.org/

In [456]:
# REGION CONNECTIVITY FROM BAMS

# Relationship CONNECTS_TO between BamsRegion and BamsRegion
csv_file_path = get_csv_file_path("regions/region_connectivity.csv")


# INPUTS AND OUTPUTS
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BamsRegion { id: row.bams_id_from})
        MATCH (c:BamsRegion { id: row.bams_id_to})
        MERGE (a)-[:CONNECTS_TO]->(c)
    """ % csv_file_path

rel_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BamsRegion {id: row.bams_id_from})-[r:CONNECTS_TO]->(b:BamsRegion { id: row.bams_id_to})
        SET r.strength = row.strength
        SET r.technique = row.technique
        SET r.description = row.description
        SET r.reference = row.reference
""" % csv_file_path


csv_file_path = get_csv_file_path("regions/bams2_mapping_regions.csv")

mapping_query = """
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BamsRegion { id: row.bams_id})
        MATCH (c:BrainRegion { id: row.bg_id})
        MERGE (a)-[:RELATES_TO]->(c)
    """ % csv_file_path

mapping_rel_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BamsRegion {id: row.bams_id})-[r:RELATES_TO]->(b:BrainRegion {id: row.bg_id})
        SET r.relationType = row.relation_type
        SET r.description = row.description
""" % csv_file_path


with driver.session() as session:
    session.run(query)
    session.run(rel_query)
    get_count_of_relationship("CONNECTS_TO", session)
    session.run(mapping_query)
    session.run(mapping_rel_query)
    get_count_of_relationship("RELATES_TO", session)

    #TODO Maybe add relation on BrainRegion-[Connects_To]-> BamsRegion and [Connects_From]
    # and in website derive it trough BAMSRegion-:RELATES_TO-> BrainRegion

Added 270 relationships of type CONNECTS_TO
Added 300 relationships of type RELATES_TO


### Morphologies from NeuroMorpho.Org
This adds morphology information to the cell types, collected from http://neuromorpho.org/
Not added for graph analytics (visualization through Gephi).

## Analysis similarity
This adds the similarity measure between all the analyses.
The simlarity is found using Neo4j graph data science library, running the node similarity on a projection of all analyses separated on specie.

As this comes from analytics, it is not added for analytics.